In [1]:
import pandas as pd
import numpy as np
import chardet as chardet
        

In [2]:

with open('Data/BHCF20250331.txt', 'rb') as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        print(f"Detected encoding: {encoding}")

Detected encoding: ascii


In [4]:
raw_data = open('Data/BHCF20250331.txt', 'rb').read()
result = chardet.detect(raw_data)
encoding = result['encoding']
print(f"Detected encoding: {encoding}")

Detected encoding: ascii


In [14]:
import glob as glob
all_files = glob.glob('Data/BHCF*.txt')

df_list = []
for file_path in all_files:
    print(file_path)
    raw_data = open(file_path, 'rb').read()
    result = chardet.detect(raw_data)
    encoding = result['encoding']
    df = pd.read_csv(file_path, delimiter = '^', encoding=encoding, engine='python', on_bad_lines='skip')
    df_list.append(df)
    print(encoding,df.shape)
combined_df = pd.concat(df_list, ignore_index=True)

Data/BHCF20250630 3.txt
ascii (3749, 2224)
Data/BHCF20231231.txt
ascii (3813, 2279)
Data/BHCF20230930.txt
ascii (448, 2279)
Data/BHCF20240630.txt
ascii (3829, 2224)
Data/BHCF20240331.txt
ascii (454, 2224)
Data/BHCF20250331.txt
ascii (462, 2224)
Data/BHCF20240930.txt
ascii (446, 2224)
Data/BHCF20241231.txt
ISO-8859-1 (3804, 2224)
Data/BHCF20230630.txt
ascii (3887, 2328)
Data/BHCF20250930.txt
ascii (55, 2224)
Data/BHCF20230331.txt
ascii (444, 2328)


In [15]:
print(combined_df.shape)

(21391, 2328)
